In [5]:
# Read Data
import numpy as np
import pandas as pd
raw_data = pd.read_csv("data_10_states_temp_precip.csv")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 6 columns):
State            1234 non-null object
Month            1234 non-null object
Year             1234 non-null int64
Crop Yield       1234 non-null float64
Temperature      1234 non-null float64
Precipitation    1234 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 57.9+ KB


In [6]:
from pandas.plotting import scatter_matrix

attributes = ["Temperature", "Precipitation", "Crop Yield"]
scatter_matrix(raw_data[attributes])

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fd47f24cf98>,
      dtype=object)

In [7]:
# split into test and training data
from sklearn.model_selection import train_test_split

raw_train_set, raw_test_set = train_test_split(raw_data, test_size = 0.2)

In [8]:
# data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

num_attribs = ["Temperature", "Precipitation"]
cat_attribs = ["Month"]

full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(sparse=False), cat_attribs),
    ])

X_train = full_pipeline.fit_transform(raw_train_set)
Y_train = raw_train_set["Crop Yield"].values

X_test = full_pipeline.fit_transform(raw_test_set)
Y_test = raw_test_set["Crop Yield"].values


In [9]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

lin_reg.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [10]:
from sklearn.metrics import mean_squared_error

lin_reg_predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(Y_test, lin_reg_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

8.276931463375657

In [11]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [12]:
tree_reg_predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(Y_test, tree_reg_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

11.290424447757768

In [33]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators = 90, random_state = 42)
forest_reg.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [34]:
forest_reg_predictions = forest_reg.predict(X_test)
forest_mse = mean_squared_error(Y_test, forest_reg_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

8.765544483107211

In [13]:
from sklearn.svm import SVR
svm_reg = SVR(kernel='linear')
svm_reg.fit(X_train, Y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
svm_reg_predictions = svm_reg.predict(X_test)
svm_mse = mean_squared_error(Y_test, svm_reg_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

8.334714258247185

In [73]:
svm_poly_reg = SVR(kernel='poly',degree = 2, C=120,epsilon = 0.3, gamma = 'auto')
svm_poly_reg.fit(X_train, Y_train)

poly_reg_predictions = svm_poly_reg.predict(X_test)
poly_mse = mean_squared_error(Y_test, poly_reg_predictions)
poly_rmse = np.sqrt(poly_mse)
poly_rmse


8.160235723497438

In [160]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.constraints import maxnorm
from keras import optimizers


nn_model = Sequential()
layer1 = 500
layer2 = 500
layer3 = 500

#nn_model.add(Dropout(0.2, input_shape=(X_train.shape[1],)))
nn_model.add(Dense(layer1, kernel_initializer='normal', input_dim=X_train.shape[1], activation='relu', kernel_constraint=maxnorm(3)))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(layer2, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(layer3, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

optimizer = optimizers.adam(lr=0.01)
nn_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
nn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_116 (Dense)            (None, 500)               3500      
_________________________________________________________________
dropout_60 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 500)               250500    
_________________________________________________________________
dropout_61 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_118 (Dense)            (None, 500)               250500    
_________________________________________________________________
dropout_62 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_119 (Dense)            (None, 1)                 501       
Total para

In [161]:
from keras.callbacks import ModelCheckpoint

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
nn_model.fit(X_train, Y_train, epochs=500, batch_size=8, validation_split = 0.2, callbacks=callbacks_list)

Train on 789 samples, validate on 198 samples
Epoch 1/500
789/789 [==============================] - 2s 2ms/step - loss: 70.4533 - mean_squared_error: 70.4533 - val_loss: 121.9306 - val_mean_squared_error: 121.9306

Epoch 00001: val_loss did not improve from 88.64744
Epoch 2/500
789/789 [==============================] - 1s 2ms/step - loss: 78.1237 - mean_squared_error: 78.1237 - val_loss: 97.5048 - val_mean_squared_error: 97.5048

Epoch 00002: val_loss did not improve from 88.64744
Epoch 3/500
789/789 [==============================] - 1s 2ms/step - loss: 71.2170 - mean_squared_error: 71.2170 - val_loss: 98.9077 - val_mean_squared_error: 98.9077

Epoch 00003: val_loss did not improve from 88.64744
Epoch 4/500
789/789 [==============================] - 2s 2ms/step - loss: 70.0114 - mean_squared_error: 70.0114 - val_loss: 100.6666 - val_mean_squared_error: 100.6666

Epoch 00004: val_loss did not improve from 88.64744
Epoch 5/500
789/789 [==============================] - 1s 2ms/step - l

789/789 [==============================] - 1s 2ms/step - loss: 63.4410 - mean_squared_error: 63.4410 - val_loss: 97.0814 - val_mean_squared_error: 97.0814

Epoch 00037: val_loss did not improve from 87.43053
Epoch 38/500
789/789 [==============================] - 1s 2ms/step - loss: 64.1490 - mean_squared_error: 64.1490 - val_loss: 89.9161 - val_mean_squared_error: 89.9161

Epoch 00038: val_loss did not improve from 87.43053
Epoch 39/500
789/789 [==============================] - 1s 2ms/step - loss: 63.7235 - mean_squared_error: 63.7235 - val_loss: 91.3550 - val_mean_squared_error: 91.3550

Epoch 00039: val_loss did not improve from 87.43053
Epoch 40/500
789/789 [==============================] - 1s 2ms/step - loss: 63.6755 - mean_squared_error: 63.6755 - val_loss: 93.9236 - val_mean_squared_error: 93.9236

Epoch 00040: val_loss did not improve from 87.43053
Epoch 41/500
789/789 [==============================] - 1s 2ms/step - loss: 62.3735 - mean_squared_error: 62.3735 - val_loss: 96.

789/789 [==============================] - 1s 2ms/step - loss: 64.9466 - mean_squared_error: 64.9466 - val_loss: 90.9548 - val_mean_squared_error: 90.9548

Epoch 00074: val_loss did not improve from 87.43053
Epoch 75/500
789/789 [==============================] - 1s 2ms/step - loss: 64.6885 - mean_squared_error: 64.6885 - val_loss: 90.8712 - val_mean_squared_error: 90.8712

Epoch 00075: val_loss did not improve from 87.43053
Epoch 76/500
789/789 [==============================] - 1s 2ms/step - loss: 66.2064 - mean_squared_error: 66.2064 - val_loss: 98.1941 - val_mean_squared_error: 98.1941

Epoch 00076: val_loss did not improve from 87.43053
Epoch 77/500
789/789 [==============================] - 1s 2ms/step - loss: 65.7272 - mean_squared_error: 65.7272 - val_loss: 97.4178 - val_mean_squared_error: 97.4178

Epoch 00077: val_loss did not improve from 87.43053
Epoch 78/500
789/789 [==============================] - 1s 2ms/step - loss: 65.7486 - mean_squared_error: 65.7486 - val_loss: 94.

789/789 [==============================] - 1s 1ms/step - loss: 64.6123 - mean_squared_error: 64.6123 - val_loss: 95.7522 - val_mean_squared_error: 95.7522

Epoch 00111: val_loss did not improve from 87.43053
Epoch 112/500
789/789 [==============================] - 1s 1ms/step - loss: 65.1833 - mean_squared_error: 65.1833 - val_loss: 92.1644 - val_mean_squared_error: 92.1644

Epoch 00112: val_loss did not improve from 87.43053
Epoch 113/500
789/789 [==============================] - 1s 1ms/step - loss: 65.0389 - mean_squared_error: 65.0389 - val_loss: 101.1231 - val_mean_squared_error: 101.1231

Epoch 00113: val_loss did not improve from 87.43053
Epoch 114/500
789/789 [==============================] - 1s 1ms/step - loss: 60.9727 - mean_squared_error: 60.9727 - val_loss: 95.4471 - val_mean_squared_error: 95.4471

Epoch 00114: val_loss did not improve from 87.43053
Epoch 115/500
789/789 [==============================] - 1s 1ms/step - loss: 64.1446 - mean_squared_error: 64.1446 - val_los

789/789 [==============================] - 1s 1ms/step - loss: 62.2320 - mean_squared_error: 62.2320 - val_loss: 92.8080 - val_mean_squared_error: 92.8080

Epoch 00148: val_loss did not improve from 87.43053
Epoch 149/500
789/789 [==============================] - 1s 1ms/step - loss: 64.8790 - mean_squared_error: 64.8790 - val_loss: 98.8403 - val_mean_squared_error: 98.8403

Epoch 00149: val_loss did not improve from 87.43053
Epoch 150/500
789/789 [==============================] - 1s 1ms/step - loss: 66.2209 - mean_squared_error: 66.2209 - val_loss: 97.5848 - val_mean_squared_error: 97.5848

Epoch 00150: val_loss did not improve from 87.43053
Epoch 151/500
789/789 [==============================] - 1s 1ms/step - loss: 62.9034 - mean_squared_error: 62.9034 - val_loss: 93.2862 - val_mean_squared_error: 93.2862

Epoch 00151: val_loss did not improve from 87.43053
Epoch 152/500
789/789 [==============================] - 1s 1ms/step - loss: 64.2689 - mean_squared_error: 64.2689 - val_loss:

...

789/789 [==============================] - 1s 1ms/step - loss: 61.2115 - mean_squared_error: 61.2115 - val_loss: 97.7674 - val_mean_squared_error: 97.7674

Epoch 00259: val_loss did not improve from 87.43053
Epoch 260/500
789/789 [==============================] - 1s 1ms/step - loss: 61.5271 - mean_squared_error: 61.5271 - val_loss: 96.9093 - val_mean_squared_error: 96.9093

Epoch 00260: val_loss did not improve from 87.43053
Epoch 261/500
789/789 [==============================] - 1s 1ms/step - loss: 60.8319 - mean_squared_error: 60.8319 - val_loss: 91.8355 - val_mean_squared_error: 91.8355

Epoch 00261: val_loss did not improve from 87.43053
Epoch 262/500
789/789 [==============================] - 1s 1ms/step - loss: 63.9692 - mean_squared_error: 63.9692 - val_loss: 92.1773 - val_mean_squared_error: 92.1773

Epoch 00262: val_loss did not improve from 87.43053
Epoch 263/500
789/789 [==============================] - 1s 1ms/step - loss: 59.3812 - mean_squared_error: 59.3812 - val_loss:


Epoch 00295: val_loss did not improve from 84.59750
Epoch 296/500
789/789 [==============================] - 1s 1ms/step - loss: 63.4431 - mean_squared_error: 63.4431 - val_loss: 97.8502 - val_mean_squared_error: 97.8502

Epoch 00296: val_loss did not improve from 84.59750
Epoch 297/500
789/789 [==============================] - 1s 1ms/step - loss: 65.8988 - mean_squared_error: 65.8988 - val_loss: 93.9277 - val_mean_squared_error: 93.9277

Epoch 00297: val_loss did not improve from 84.59750
Epoch 298/500
789/789 [==============================] - 1s 1ms/step - loss: 61.5500 - mean_squared_error: 61.5500 - val_loss: 101.2116 - val_mean_squared_error: 101.2116

Epoch 00298: val_loss did not improve from 84.59750
Epoch 299/500
789/789 [==============================] - 1s 1ms/step - loss: 65.4249 - mean_squared_error: 65.4249 - val_loss: 97.3818 - val_mean_squared_error: 97.3818

Epoch 00299: val_loss did not improve from 84.59750
Epoch 300/500
789/789 [==============================] - 


Epoch 00332: val_loss did not improve from 84.59750
Epoch 333/500
789/789 [==============================] - 1s 2ms/step - loss: 61.1665 - mean_squared_error: 61.1665 - val_loss: 96.0759 - val_mean_squared_error: 96.0759

Epoch 00333: val_loss did not improve from 84.59750
Epoch 334/500
789/789 [==============================] - 1s 1ms/step - loss: 63.0084 - mean_squared_error: 63.0084 - val_loss: 100.3248 - val_mean_squared_error: 100.3248

Epoch 00334: val_loss did not improve from 84.59750
Epoch 335/500
789/789 [==============================] - 1s 1ms/step - loss: 63.1371 - mean_squared_error: 63.1371 - val_loss: 95.7280 - val_mean_squared_error: 95.7280

Epoch 00335: val_loss did not improve from 84.59750
Epoch 336/500
789/789 [==============================] - 1s 1ms/step - loss: 60.6068 - mean_squared_error: 60.6068 - val_loss: 96.6107 - val_mean_squared_error: 96.6107

Epoch 00336: val_loss did not improve from 84.59750
Epoch 337/500
789/789 [==============================] - 


Epoch 00369: val_loss did not improve from 84.59750
Epoch 370/500
789/789 [==============================] - 1s 1ms/step - loss: 60.6674 - mean_squared_error: 60.6674 - val_loss: 93.7575 - val_mean_squared_error: 93.7575

Epoch 00370: val_loss did not improve from 84.59750
Epoch 371/500
789/789 [==============================] - 1s 1ms/step - loss: 63.7163 - mean_squared_error: 63.7163 - val_loss: 92.6095 - val_mean_squared_error: 92.6095

Epoch 00371: val_loss did not improve from 84.59750
Epoch 372/500
789/789 [==============================] - 1s 1ms/step - loss: 60.2708 - mean_squared_error: 60.2708 - val_loss: 92.6379 - val_mean_squared_error: 92.6379

Epoch 00372: val_loss did not improve from 84.59750
Epoch 373/500
789/789 [==============================] - 1s 1ms/step - loss: 61.9834 - mean_squared_error: 61.9834 - val_loss: 94.8193 - val_mean_squared_error: 94.8193

Epoch 00373: val_loss did not improve from 84.59750
Epoch 374/500
789/789 [==============================] - 1s


Epoch 00406: val_loss did not improve from 84.59750
Epoch 407/500
789/789 [==============================] - 1s 2ms/step - loss: 62.1577 - mean_squared_error: 62.1577 - val_loss: 90.1317 - val_mean_squared_error: 90.1317

Epoch 00407: val_loss did not improve from 84.59750
Epoch 408/500
789/789 [==============================] - 1s 2ms/step - loss: 63.5750 - mean_squared_error: 63.5750 - val_loss: 92.7080 - val_mean_squared_error: 92.7080

Epoch 00408: val_loss did not improve from 84.59750
Epoch 409/500
789/789 [==============================] - 1s 2ms/step - loss: 59.7824 - mean_squared_error: 59.7824 - val_loss: 95.3908 - val_mean_squared_error: 95.3908

Epoch 00409: val_loss did not improve from 84.59750
Epoch 410/500
789/789 [==============================] - 1s 2ms/step - loss: 63.1876 - mean_squared_error: 63.1876 - val_loss: 97.1796 - val_mean_squared_error: 97.1796

Epoch 00410: val_loss did not improve from 84.59750
Epoch 411/500
789/789 [==============================] - 2s

In [163]:
nn_predictions = nn_model.predict(X_test)
nn_mse = mean_squared_error(Y_test, nn_predictions)
nn_rmse = np.sqrt(nn_mse)
nn_rmse

8.559820635687789